In [1]:
#All Imports 
!pip install datasets
!pip install transformers
!pip install SentencePiece

# !pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

import torch
import transformers
from transformers import AutoTokenizer
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

from transformers import pipeline, set_seed
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric


from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# import nltk
# from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Transfer learning 
- train the pre-train model 

In [2]:
import pandas as pd
#mounting drive
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/MyDrive/NLP/Ass3/'

# article_df = pd.read_csv(data_dir + 'md_pmc_covid_articles.csv')
# article_df.head(5)
proc_article_df = pd.read_csv(data_dir + 'Processed_covid_articles.csv')
proc_article_df = proc_article_df.drop(['Unnamed: 0'], axis=1)
proc_article_df.head(5)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,paper_id,body_text,authors,title,abstract,Language
0,PMC8524215,cancer prevalent cause mortality world common ...,"['Muhammad Sufyan ', 'Farah Shahid ', 'Faiza...",implementation vaccinomics insilico approach c...,68591 One of the most common gynecologic ca...,en
1,PMC7698314,sarscov2 virus responsible covid19 disease fir...,"['Enrique Casalino ', 'Christophe Choquet ',...",analysis emergency department visit hospital a...,813557 ED-visits and through-ED admissions ...,en
2,PMC8050998,artificial intelligence ai methodology compute...,"['Martina Gurgitano ', 'Salvatore Alessio Ang...",interventional radiology exmachina impact arti...,757155 Artificial intelligence (AI) is a br...,en
3,PMC7581283,common feature patient admitted coronavirus di...,"['Ejvind Frausing Hansen ', 'Charlotte Sandau ...",automatic oxygen titration o2matic® patient ad...,942009 INTRODUCTION: Patients with coronavi...,en
4,PMC7392455,coronavirus disease covid19 caused severe acut...,"['Daniel O. Griffin ', 'Alexandra Jensen ', '...",pulmonary embolism increased level ddimer pati...,843714 We report 3 patients with coronaviru...,en


In [3]:
#Showing a sample 
sample = proc_article_df.iloc[0]
print(f"""
Article (excerpt of 500 characters, total length: {len(sample["body_text"])}):
""")
print(sample["body_text"][:500])
print(f'\nSummary (length: {len(sample["abstract"])}):')
print(sample["abstract"])


Article (excerpt of 500 characters, total length: 25277):

cancer prevalent cause mortality world common hindrance country achieving desired life expectancy . ovarian cancer one common type cancer woman . guleria . 2020 . ovarian cancer extremely lethal nastiest prognosis ailment . inability early diagnosis nearly nonspecific symptom common ovarian cancer identified iiiiv stage . five major histological form ovarian cancer identified epithelial edward . 2016 torre . 2018 siegel miller 2016 . endometrioid 13 mucinous 13 clear cell cancer 6 three prevalen

Summary (length: 88):
68591    One of the most common gynecologic cancers is ...
Name: abstract, dtype: object


In [4]:
import pandas as pd
# divide the dataframe into training data and test data
article_df_copy = proc_article_df.copy()
train_article = article_df_copy[['body_text','abstract']].iloc[:4000]  # 4000 rows for training
test_article = article_df_copy[['body_text','abstract']].iloc[4000:] # 500 rows for testing

train_article.head(5)

,body_text,abstract
0,cancer prevalent cause mortality world common ...,68591 One of the most common gynecologic ca...
1,sarscov2 virus responsible covid19 disease fir...,813557 ED-visits and through-ED admissions ...
2,artificial intelligence ai methodology compute...,757155 Artificial intelligence (AI) is a br...
3,common feature patient admitted coronavirus di...,942009 INTRODUCTION: Patients with coronavi...
4,coronavirus disease covid19 caused severe acut...,843714 We report 3 patients with coronaviru...


In [5]:
def tokenization(df,tokenizer):
    
    # tokenize in ids the body text and abstract
    tokenized_output = list()
    for i,j in zip(df['body_text'], df['abstract']):
        f_tokenized = tokenizer(i, padding="max_length", truncation=True)
        l_tokenized = tokenizer(j, padding="max_length", truncation=True)
        tokenized_text ={ 
        'feature_input_ids': f_tokenized['input_ids'].flatten(),
        'feature_attention_mask': f_tokenized['attention_mask'],
        'feature_text': i,
        'label_input_ids': l_tokenized['input_ids'].flatten(),
        'label_attention_mask': l_tokenized['attention_mask'],
        'label_text': j
        }
        
        tokenized_output.append(tokenized_text)
    return tokenized_output


## T5 model 

In [ ]:
import transformers

model_name = 't5-small'
model = transformers.AutoModelWithLMHead.from_pretrained(model_name)

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

nlp_summarizer = transformers.pipeline('summarization', model = model, tokenizer = tokenizer)

nlp_summarizer(sample['body_text'])



/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:1295: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding

[{'summary_text': 'ovarian cancer one common type of cancer identified iiiv stage . inability early diagnosis nearly nonspecific symptom common type cancer . hla allele significant since hold response .'}]

In [ ]:
# import torch
# import transformers

# model_name = 't5-small'
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# t5_model = transformers.AutoModelWithLMHead.from_pretrained(model_name).to(device)
# t5_tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

# input_text = sample["body_text"]
# input_ids = t5_tokenizer.encode(input_text, truncation=True, padding="longest", return_tensors='pt', max_length=500).to(device)
# nlp_summarizer = transformers.pipeline('summarization', model=t5_model, tokenizer=t5_tokenizer, device=device.index if torch.cuda.is_available() else -1)
# output = nlp_summarizer(input_text)

## Pegasus Model

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

src_text = sample['body_text']

model_name = "google/pegasus-xsum"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
batch = tokenizer(src_text, truncation=True, padding="longest", return_tensors="pt").to(device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
print(tgt_text)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (64) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['ovarian cancer one common type cancer woman.']


In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

src_text = sample['body_text']

model_name = "google/pegasus-large"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer = PegasusTokenizer.from_pretrained(model_name)

model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
batch = tokenizer(src_text, truncation=True, padding="longest", return_tensors="pt",max_length= 500).to(device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
print(tgt_text)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (256) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['endometrioid 13 mucinous 13 clear cell cancer 6 three prevalent type ovarian cancer coburn. benign ovarian tumor recommended threat indicator precursor epithelial ovarian cancer ovarian cancer benign ovarian tumor share equivalent etiological risk factor jordan sj green ac whiteman dc webb pm study ac 2007. clear cell endometrioid tumor wellthoughtout caused establishing endometrium ovary throughout retrograde menstruation kurman 2016. different reason predicting greater mortality rate ovarian cancer like nonsymptomatic stealthy progression malignant tumorous cell slow progression disease symptom lastly lacking specific method diagnosis cancer even advanced stage. 4.1 per 100000 compared 21290 case 14180 death related ovarian cancer usa 2015.']


In [ ]:
sample['body_text']

'cancer prevalent cause mortality world common hindrance country achieving desired life expectancy . ovarian cancer one common type cancer woman . guleria . 2020 . ovarian cancer extremely lethal nastiest prognosis ailment . inability early diagnosis nearly nonspecific symptom common ovarian cancer identified iiiiv stage . five major histological form ovarian cancer identified epithelial edward . 2016 torre . 2018 siegel miller 2016 . endometrioid 13 mucinous 13 clear cell cancer 6 three prevalent type ovarian cancer coburn . 2017 . benign ovarian tumor recommended threat indicator precursor epithelial ovarian cancer ovarian cancer benign ovarian tumor share equivalent etiological risk factor jordan sj green ac whiteman dc webb pm study ac 2007 . also proposed fallopian tube may also contribute origination ovarian cancer distal end fallopian tube predecessor lesion ovary could progress severe ovarian tumor kurman 2016 . clear cell endometrioid tumor wellthoughtout caused establishing e

## Fine Tuning

In [6]:
sm_train_article = train_article.iloc[:1000]
sm_train_article.head()

,body_text,abstract
0,cancer prevalent cause mortality world common ...,68591 One of the most common gynecologic ca...
1,sarscov2 virus responsible covid19 disease fir...,813557 ED-visits and through-ED admissions ...
2,artificial intelligence ai methodology compute...,757155 Artificial intelligence (AI) is a br...
3,common feature patient admitted coronavirus di...,942009 INTRODUCTION: Patients with coronavi...
4,coronavirus disease covid19 caused severe acut...,843714 We report 3 patients with coronaviru...


### Fine Tuning Bert

In [11]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

### convert to Huggingface dataset
sm_train_db = Dataset(pa.Table.from_pandas(sm_train_article))

In [14]:
from transformers import BartTokenizer
from torch.utils.data import DataLoader
model_name = 't5-small'
article_length = 512
summary_length = 64
batch_size     = 4

# Load the BART's pre-trained Tokenizer
# TODO use pretrained model for medical data, covid would be best
# tokenizer = BartTokenizer.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
# Define the function to make the correct data structure
def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  inputs = tokenizer(batch["body_text"], padding="max_length", truncation=True, max_length=article_length)
  outputs = tokenizer(batch["abstract"], padding="max_length", truncation=True, max_length=summary_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["decoder_attention_mask"] = outputs.attention_mask
  batch["labels"] = outputs.input_ids.copy()

  # We have to make sure that the PAD token is ignored for calculating the loss
  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch

# Map the function to both train/validation sets.
train_data = sm_train_db.map(
    process_data_to_model_inputs, 
    batched=True,
    remove_columns=["body_text", "abstract"]
)

# Convert the Dataset to PyTorch tensor with the expected columns
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids",
                           "decoder_attention_mask", "labels"],
)

# Make the iterative object that does batching using the DataLoader
train_data = DataLoader(train_data, batch_size=batch_size)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [17]:
from transformers import BartForConditionalGeneration
import torch
#https://pub.towardsai.net/how-to-train-a-seq2seq-text-summarization-model-with-sample-code-ft-huggingface-pytorch-8ba97492f885
# Load the model
# model = BartForConditionalGeneration.from_pretrained(model_name)
b_model = transformers.AutoModelWithLMHead.from_pretrained(model_name)

# Put the model on GPU
if torch.cuda.is_available():
  b_model = b_model.to("cuda")
  print('using gpu')

# Split model's components
the_encoder = b_model.get_encoder()
the_decoder = b_model.get_decoder()
last_linear_layer = b_model.lm_head

using gpu


In [18]:
from torch.nn import CrossEntropyLoss
from transformers import AdamW
from transformers import get_scheduler

num_epochs = 20
num_training_steps = num_epochs * len(train_data)

# The loss function
loss_fct = CrossEntropyLoss()

# The optimizer
optimizer = AdamW(b_model.parameters(), lr=0.001)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
for epoch in range(num_epochs):
    print('epoch number ', epoch)
    b_model.train()
    for batch in train_data:
      if torch.cuda.is_available():
        batch = {k: v.to('cuda') for k, v in batch.items()}

      # Get the "input's representation"
      encoder_output = the_encoder(input_ids = batch['input_ids'],
                                   attention_mask = batch['attention_mask'])
      
      # Pass the representation + the target summary to the decoder
      decoder_output = the_decoder(input_ids=batch['decoder_input_ids'],
                                   attention_mask=batch['decoder_attention_mask'],
                                   encoder_hidden_states=encoder_output[0],
                                   encoder_attention_mask=batch['attention_mask'])

      # Use the last linear layer to predict the next token
      decoder_output = decoder_output.last_hidden_state
      lm_head_output = last_linear_layer(decoder_output)
      
      # Compute the loss
      loss = loss_fct(lm_head_output.view(-1, b_model.config.vocab_size),
                      batch['labels'].view(-1))
      
      loss.backward() # Update the weights
      optimizer.step() # Notify optimizer that a batch is done.
      lr_scheduler.step() # Notify the scheduler that a ...
      optimizer.zero_grad() # Reset the optimer


print('End of training')

epoch number  0
epoch number  1
epoch number  2
epoch number  3
epoch number  4
epoch number  5
epoch number  6
epoch number  7
epoch number  8
epoch number  9
epoch number  10
epoch number  11
epoch number  12
epoch number  13
epoch number  14
epoch number  15
epoch number  16
epoch number  17
epoch number  18
epoch number  19
End of training


In [22]:
validation_data = Dataset(pa.Table.from_pandas(train_article.iloc[:1000]))
### convert to Huggingface dataset

validation_data = validation_data.map(
    process_data_to_model_inputs, 
    batched=True,
    remove_columns=["body_text", "abstract"]
)

# Convert the Dataset to PyTorch tensor with the expected columns
validation_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids",
                           "decoder_attention_mask", "labels"],
)

# Make the iterative object that does batching using the DataLoader
validation_data = DataLoader(validation_data, batch_size=batch_size)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [25]:
b_model.eval()
for batch in validation_data:
    if torch.cuda.is_available():
        # print('batch ', batch)
        # for k, v in batch.items():
            # print('k ', k)
            # print('v ', v)
        batch = {k: v.to('cuda') for k, v in batch.items()}
        
    with torch.no_grad():
        outputs = b_model(**batch)

    # print('outputs ', outputs)
    loss = outputs.loss
    # print('loss ', loss)
    # fds

In [31]:
ARTICLE_TO_SUMMARIZE = (train_article.iloc[0]['body_text'])

inputs = tokenizer(ARTICLE_TO_SUMMARIZE, return_tensors="pt")

if torch.cuda.is_available():
    inputs = {k: v.to('cuda') for k, v in inputs.items()}

In [34]:
inputs

{'input_ids': tensor([[ 1874, 22519,  1137,  ...,     3,     5,     1]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0')}

In [36]:
# Generate Summary using custom trained model
summary_ids = b_model.generate(inputs["input_ids"], num_beams=100, min_length=0, max_length=500)
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

OutOfMemoryError: ignored

### Fine Tuning Pegasus

In [8]:
### convert to Huggingface dataset
sm_train_db = Dataset(pa.Table.from_pandas(sm_train_article))

In [ ]:
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", truncation=True)


# tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [9]:
from transformers import AutoTokenizer
from torch.utils.data import DataLoader

article_length = 512
summary_length = 64
batch_size     = 4
model_name = "google/pegasus-large"
# Load the BART's pre-trained Tokenizer
# TODO use pretrained model for medical data, covid would be best
# tokenizer = BartTokenizer.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  inputs = tokenizer(batch["body_text"], padding="max_length", truncation=True, max_length=article_length)
  outputs = tokenizer(batch["abstract"], padding="max_length", truncation=True, max_length=summary_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  # batch["input_text"] = batch["body_text"]
  batch["decoder_input_ids"] = outputs.input_ids
  batch["decoder_attention_mask"] = outputs.attention_mask
  # batch["decoder_text"] = batch["abstract"]
  batch["labels"] = outputs.input_ids.copy()

  # We have to make sure that the PAD token is ignored for calculating the loss
  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch

# Map the function to both train/validation sets.
train_data = sm_train_db.map(
    process_data_to_model_inputs, 
    batched=True,
    remove_columns=["body_text", "abstract"]
)

# Convert the Dataset to PyTorch tensor with the expected columns
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids",
                           "decoder_attention_mask", "labels"],
)

# Make the iterative object that does batching using the DataLoader
train_data      = DataLoader(train_data, batch_size=batch_size)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [25]:
# for i in train_data:
#   print(i)

In [10]:
from transformers import PegasusForConditionalGeneration

model_name = "google/pegasus-large"
model = transformers.PegasusForConditionalGeneration.from_pretrained(model_name)

# Put the model on GPU
if torch.cuda.is_available():
    model.to("cuda")
    print("Using GPU")

# Split model's components
the_encoder = model.get_encoder()
the_decoder = model.get_decoder()
last_linear_layer = model.lm_head

Using GPU


In [11]:
from torch.nn import CrossEntropyLoss
from transformers import AdamW
from transformers import get_scheduler

num_epochs = 20
num_training_steps = num_epochs * len(train_data)

# The loss function
loss_fct = CrossEntropyLoss()

# The optimizer
optimizer = AdamW(model.parameters(), lr=0.001)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
for epoch in range(num_epochs):
    print('epoch number ', epoch)
    model.train()
    for batch in train_data:
      if torch.cuda.is_available():
        batch = {k: v.to('cuda') for k, v in batch.items()}

      # Get the "input's representation"
      encoder_output = the_encoder(input_ids = batch['input_ids'],
                                   attention_mask = batch['attention_mask'])
      
      # Pass the representation + the target summary to the decoder
      decoder_output = the_decoder(input_ids=batch['decoder_input_ids'],
                                   attention_mask=batch['decoder_attention_mask'],
                                   encoder_hidden_states=encoder_output[0],
                                   encoder_attention_mask=batch['attention_mask'])

      # Use the last linear layer to predict the next token
      decoder_output = decoder_output.last_hidden_state
      # lm_head_output = last_linear_layer(decoder_output)
      lm_head_output = last_linear_layer(decoder_output)
      
      # Compute the loss
      loss = loss_fct(lm_head_output.view(-1, model.config.vocab_size),
                      batch['labels'].view(-1))
      
      loss.backward() # Update the weights
      optimizer.step() # Notify optimizer that a batch is done.
      lr_scheduler.step() # Notify the scheduler that a ...
      optimizer.zero_grad() # Reset the optimer


print('End of training')

epoch number  0
epoch number  1
epoch number  2
epoch number  3
epoch number  4
epoch number  5
epoch number  6
epoch number  7
epoch number  8


In [13]:
model.eval()
ARTICLE_TO_SUMMARIZE = (train_article.iloc[0]['body_text'])
inputs = tokenizer([ARTICLE_TO_SUMMARIZE], return_tensors="pt")
if torch.cuda.is_available():
    inputs = {k: v.to('cuda') for k, v in inputs.items()}
# Generate Summary using custom trained model
summary_ids = model.generate(inputs["input_ids"], num_beams=2, min_length=0, max_length=500)
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

RuntimeError: ignored

RuntimeError: ignored

In [12]:
ARTICLE_TO_SUMMARIZE = (train_article.iloc[0]['body_text'])
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Tokenize the input article
inputs = tokenizer(ARTICLE_TO_SUMMARIZE, return_tensors="pt")

# Move the input tensor to the same device as the model
inputs = {k: v.to(device) for k, v in inputs.items()}

# Generate Summary using custom trained model
summary_ids = model.generate(inputs["input_ids"], num_beams=2, min_length=0, max_length=500)

# Decode the summary
summary = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]


Token indices sequence length is longer than the specified maximum sequence length for this model (4975 > 1024). Running this sequence through the model will result in indexing errors


RuntimeError: ignored

In [11]:

from torch.utils.data import DataLoader
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, 
                               batch_size=16, device=device, 
                               column_text="body_text", 
                               column_summary="abstract"):
    article_batches = DataLoader(article_batches[column_text], batch_size=batch_size)
    target_batches = DataLoader(article_batches[column_summary], batch_size=batch_size)
    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
        
        inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                        padding="max_length", return_tensors="pt")
        
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device), 
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
        
        # Finally, we decode the generated texts, 
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                clean_up_tokenization_spaces=True) 
               for s in summaries]      
        
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
        
        
        metric.add_batch(predictions=decoded_summaries, references=target_batch)
        
    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [12]:
model_goog_peg_lrg = "google/pegasus-large"
pipe = pipeline('summarization', model = model_goog_peg_lrg )

pipe_out = pipe(train_article.iloc[0]['body_text'][:1024])

print(pipe_out)

ValueError: ignored

In [ ]:
train_article.iloc[0]['body_text']

'cancer prevalent cause mortality world common hindrance country achieving desired life expectancy . ovarian cancer one common type cancer woman . guleria . 2020 . ovarian cancer extremely lethal nastiest prognosis ailment . inability early diagnosis nearly nonspecific symptom common ovarian cancer identified iiiiv stage . five major histological form ovarian cancer identified epithelial edward . 2016 torre . 2018 siegel miller 2016 . endometrioid 13 mucinous 13 clear cell cancer 6 three prevalent type ovarian cancer coburn . 2017 . benign ovarian tumor recommended threat indicator precursor epithelial ovarian cancer ovarian cancer benign ovarian tumor share equivalent etiological risk factor jordan sj green ac whiteman dc webb pm study ac 2007 . also proposed fallopian tube may also contribute origination ovarian cancer distal end fallopian tube predecessor lesion ovary could progress severe ovarian tumor kurman 2016 . clear cell endometrioid tumor wellthoughtout caused establishing e

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

rouge_metric = load_metric('rouge')

score = calculate_metric_on_test_ds(train_article, rouge_metric, model_pegasus, tokenizer, column_text = 'body_text', column_summary='abstract')

k


## Rough

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
temp = tokenizer(train_article['body_text'][0], padding="max_length", truncation=True)
temp.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenization(df,tokenizer):
    
    # tokenize in ids the body text and abstract
    tokenized_output = list()
    for i,j in zip(df['body_text'], df['abstract']):
        f_tokenized = tokenizer(i, padding="max_length", truncation=True)
        l_tokenized = tokenizer(j, padding="max_length", truncation=True)
        tokenized_text ={ 
        'feature_input_ids': f_tokenized['input_ids'].flatten(),
        'feature_attention_mask': f_tokenized['attention_mask'],
        'feature_text': i,
        'label_input_ids': l_tokenized['input_ids'].flatten(),
        'label_attention_mask': l_tokenized['attention_mask'],
        'label_text': j
        }
        
        tokenized_output.append(tokenized_text)
    return tokenized_output

# list of dictionaries, each dictionary is a tokenized article with 'input_ids', 'token_type_ids', 'attention_mask'

train_X_encode_ids = tokenization(train_article,tokenizer)
test_X_encode_ids = tokenization(test_article,tokenizer)

# train_y_encode_ids = tokenization(train_article, "abstract")
# test_y_encode_ids = tokenization(test_article, "abstract")



In [ ]:
train_X_encode_ids[0].keys()

dict_keys(['feature_input_ids', 'feature_attention_mask', 'feature_text', 'label_input_ids', 'label_attention_mask', 'label_text'])

In [ ]:
small_train_dataset = train_X_encode_ids[0:1000]
small_eval_dataset = test_X_encode_ids[0:1000]

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

In [ ]:
from transformers import TFAutoModelForSequenceClassification
from transformers import BertForSequenceClassification
import torch
from torch.optim import Adam
from transformers import get_scheduler
import tqdm

l_rate = 1e-5
epochs = 2
train_steps = epochs * len(train_dataloader)
#Get transformaer specific model later
t_model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
optimizer = Adam(params = t_model.parameters(), lr=l_rate)

lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=train_steps
)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
import torch
mps_device = torch.device("cpu")
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print(x)
else:
    print("MPS device not found.")

tensor([1.], device='mps:0')


In [ ]:
def train(epochs, model, optimizer,train_dataloader):
    train_loss_ls = []
    train_accuracy_ls = []
    val_loss_ls = []
    val_accuracy_ls = []
    model = model.to(mps_device)
    #progress_bar = tqdm(range(train_steps))
    '''
     This is the training section
    '''
    
    for epoch in range(epochs):  # loop over the dataset multiple times
        
        training_loss = 0
        training_correct = 0
        model.train() # the model is in training mode, will consider evething like batch normalization and dropout
            
        for batch in train_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            #progress_bar.update(1)

train(epochs, t_model, optimizer,train_dataloader)

            
        
        # for i, data in enumerate(train_dataloader, 0): # each data is one batch
        #     # get the inputs; data is a list of [inputs, labels]
        #     inputs, labels = data
        #     inputs, labels = inputs.to(mps_device), labels.to(mps_device)

        #     # zero the parameter gradients
        #     optimizer.zero_grad()

        #     # forward + backward + optimize
        #     outputs = model(inputs)
        #     predicted = outputs.argmax(1)# index of max probability out of 10 classes

        #     loss = criterion(outputs, labels)
        #     loss.backward()
        #     optimizer.step()

        #     # print statistics
        #     training_loss += loss.item()
        #     training_correct += (predicted == labels).sum().item()

#         '''
#         This is the validation section
# #         '''
#         val_loss = 0
#         val_correct = 0
# #         val_total = 0
#         model.eval()
#         with torch.no_grad(): # do not perform any calculation  
#             for data in test_dataloader:
#                 inputs, labels = data
#                 inputs, labels = inputs.to(device), labels.to(device)
#                 outputs = model(inputs)
#                 predicted = outputs.argmax(1)
#                 loss = criterion(outputs, labels)
#                 val_loss += loss.item()
# #                 val_total += labels.size(0)
#                 val_correct += (predicted == labels).sum().item()

# #         print('Val Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

#         training_loss = training_loss/(len(trainloader_resnet)) # this is consider as batch
#         train_acc = (training_correct / len(trainloader_resnet.dataset)) # this consider as all simple
#         val_loss = val_loss / len(validloader_resnet)
#         val_acc = (val_correct/len(validloader_resnet.dataset))
        
#         # save the loss and accuracy in the list
#         train_loss_ls.append(training_loss)
#         train_accuracy_ls.append(train_acc)
#         val_loss_ls.append(val_loss)
#         val_accuracy_ls.append(val_acc)
#         print("epoch",epoch)
#         print("train_accuracy: ",train_acc,"val_accuracy: ",val_acc,"train_loss: ",training_loss, "val_loss:", val_loss)

#     print('Finished Training')
#     return (train_accuracy_ls,train_loss_ls,val_accuracy_ls,val_loss_ls)


AttributeError: 'list' object has no attribute 'to'

## Takes Too much RAM

In [7]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, Trainer, TrainingArguments
import torch


class PegasusDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])  # torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels['input_ids'])  # len(self.labels)

      
def prepare_data(model_name, 
                 train_texts, train_labels, 
                 val_texts=None, val_labels=None, 
                 test_texts=None, test_labels=None):
  """
  Prepare input data for model fine-tuning
  """
  tokenizer = PegasusTokenizer.from_pretrained(model_name)

  prepare_val = False if val_texts is None or val_labels is None else True
  prepare_test = False if test_texts is None or test_labels is None else True

  def tokenize_data(texts, labels):
    encodings = tokenizer(texts, truncation=True, padding=True)
    decodings = tokenizer(labels, truncation=True, padding=True)
    dataset_tokenized = PegasusDataset(encodings, decodings)
    return dataset_tokenized

  train_dataset = tokenize_data(train_texts, train_labels)
  val_dataset = tokenize_data(val_texts, val_labels) if prepare_val else None
  test_dataset = tokenize_data(test_texts, test_labels) if prepare_test else None

  return train_dataset, val_dataset, test_dataset, tokenizer


def prepare_fine_tuning(model_name, tokenizer, train_dataset, val_dataset=None, freeze_encoder=False, output_dir='./results'):
  """
  Prepare configurations and base model for fine-tuning
  """
  torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
  model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

  if freeze_encoder:
    for param in model.model.encoder.parameters():
      param.requires_grad = False

  if val_dataset is not None:
    training_args = TrainingArguments(
      output_dir=output_dir,           # output directory
      num_train_epochs=3,           # total number of training epochs
      per_device_train_batch_size=1,   # batch size per device during training, can increase if memory allows
      per_device_eval_batch_size=1,    # batch size for evaluation, can increase if memory allows
      save_steps=100,                  # number of updates steps before checkpoint saves
      save_total_limit=5,              # limit the total amount of checkpoints and deletes the older checkpoints
      evaluation_strategy='steps',     # evaluation strategy to adopt during training
      eval_steps=50,                  # number of update steps before evaluation
      warmup_steps=50,                # number of warmup steps for learning rate scheduler
      weight_decay=0.01,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      logging_steps=10,
    )

    trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      eval_dataset=val_dataset,            # evaluation dataset
      tokenizer=tokenizer
    )

  else:
    training_args = TrainingArguments(
      output_dir=output_dir,           # output directory
      num_train_epochs=3,           # total number of training epochs
      per_device_train_batch_size=1,   # batch size per device during training, can increase if memory allows
      save_steps=100,                  # number of updates steps before checkpoint saves
      save_total_limit=5,              # limit the total amount of checkpoints and deletes the older checkpoints
      warmup_steps=50,                # number of warmup steps for learning rate scheduler
      weight_decay=0.01,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      logging_steps=10,
    )

    trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      tokenizer=tokenizer
    )

  return trainer


if __name__=='__main__':
  # # use XSum dataset as example, with first 1000 docs as training data
  # from datasets import load_dataset
  # dataset = load_dataset("xsum")
  # train_texts, train_labels = dataset['train']['document'][:1000], dataset['train']['summary'][:1000]
  
  # use Pegasus Large model as base for fine-tuning
  model_name = 'google/pegasus-large'
  tr_f = [i for i in train_article.iloc[:100]['body_text']]
  tr_l = [i for i in train_article.iloc[:100]['abstract']]
  train_dataset, _, _, tokenizer = prepare_data(model_name, tr_f, tr_l)
  trainer = prepare_fine_tuning(model_name, tokenizer, train_dataset)
  trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,9.218700
20,8.159700
30,8.312800
40,7.424700
50,6.579300
60,5.535800
70,5.102600
80,5.009500
90,4.144500
100,4.813500


In [8]:
ARTICLE_TO_SUMMARIZE = (train_article.iloc[0]['body_text'])

tokenizer = PegasusTokenizer.from_pretrained(model_name)
inputs = tokenizer(ARTICLE_TO_SUMMARIZE, return_tensors="pt")

if torch.cuda.is_available():
    inputs = {k: v.to('cuda') for k, v in inputs.items()}

inputs

Token indices sequence length is longer than the specified maximum sequence length for this model (4975 > 1024). Running this sequence through the model will result in indexing errors


{'input_ids': tensor([[ 1695, 12273,  1007,  ...,   110,   107,     1]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0')}

In [9]:
# Generate Summary using custom trained model
summary_ids = trainer.model.generate(inputs["input_ids"], num_beams=2, min_length=0, max_length=500)
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

RuntimeError: ignored